In [7]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.applications.vgg16 import VGG16 
import numpy as np

In [48]:
img_width , img_height = 48,48
train_dir = 'data/train/'
test_dir = 'data/test/'
nb_train = 7001
nb_test = 4001
epochs = 50
batch_size = 16
input_shape = (img_width,img_height,3)

In [6]:
model = Sequential()

model.add(Conv2D(32,(3,3),input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(7))
model.add(Activation('softmax'))

model.compile(loss = 'categorical_crossentropy',
             optimizer = 'rmsprop',
             metrics = ['accuracy'])

In [7]:
train_datagen = ImageDataGenerator(
                rescale = 1./255,
                shear_range = 0.2,
                zoom_range = 0.2,
                horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
                train_dir,
                target_size = (img_width,img_height),
                batch_size = batch_size,
                class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 7001 images belonging to 7 classes.
Found 4001 images belonging to 7 classes.


In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=nb_test // batch_size)

Epoch 1/10
4375/4375 [==============================] - 453s - loss: 1.5946 - acc: 0.3760 - val_loss: 1.3695 - val_acc: 0.4802

# Transfer Learning

In [33]:
def save_features():
    datagen = ImageDataGenerator(rescale = 1./255)
    model = VGG16(weights = 'imagenet',include_top = False)
    generator = datagen.flow_from_directory(
                train_dir,
                target_size = (img_width,img_height),
                batch_size = batch_size,
                class_mode = 'categorical',
                shuffle = False)
    features_train = model.predict_generator(generator,nb_train//batch_size)
    np.save(open('features_train.npy','wb'),features_train)
    
    generator = datagen.flow_from_directory(
                test_dir,
                target_size = (img_width,img_height),
                batch_size = batch_size,
                class_mode = 'categorical',
                shuffle = False)
    features_test = model.predict_generator(generator,nb_test//batch_size)
    np.save(open('features_test.npy','wb'),features_test)

In [34]:
save_features()

Found 7001 images belonging to 7 classes.
Found 4001 images belonging to 7 classes.


In [51]:
def transfer_learning():
    train_data = np.load(open('features_train.npy','rb'))
    train_labels = np.array([0]*1000 + [1]*101 + [2]*1022 + [3]*1776 + [4]*1168 + [5]*742 + [6]*1183)
    print(train_data.shape)
    print(train_labels.shape)
    test_data = np.load(open('features_test.npy','rb'))
    test_labels = np.array([0]*529 + [1]*63 + [2]*551 + [3]*989 + [4]*718 + [5]*458 + [6]*692)
    print(test_data.shape)
    print(test_labels.shape)
    model = Sequential()
    model.add(Flatten(input_shape = train_data.shape[1:]))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7))
    model.add(Activation('softmax'))
    
    model.compile(loss = 'sparse_categorical_crossentropy',
                optimizer = 'rmsprop',
                metrics = ['accuracy'])
    
    model.fit(train_data,train_labels,
             epochs = epochs,
             batch_size = batch_size,
             validation_data = (test_data,test_labels))
    

In [ ]:
transfer_learning()

(6992, 1, 1, 512)
(6992,)
(4000, 1, 1, 512)
(4000,)
Train on 6992 samples, validate on 4000 samples
Epoch 1/50
6992/6992 [==============================] - 1s - loss: 1.7561 - acc: 0.2832 - val_loss: 1.6463 - val_acc: 0.3372